# My Approach

The problem I have chosen to solve is Task 1: Less than k

Task 1: 
Given a positive integer “k” and a list of integer numbers, look for the numbers within the list, that are less than k. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case 

From above task, I understood that I need to get result of comparison between two numbers or bitstrings of arbitary length.

I have no idea about how to go about it. After all I have only read 'Introduction to Classical and Quantum Computing' by Thomas G.Wong and a few quantum computing blogs.

My first idea was to search the classical circuit for comparing two numbers and convert it into a  reversible quantum circuit as we did in conversion of classical adder to quantum adder.

The search was mostly futile. So, I attempted the second idea.

The second idea was to directly find a quantum circuit that compares two numbers.
Voila, I found a few blog and a research paper. My solution is inspired from below mentioned references.

What I understood: \
Only when I know how to compare just two single-bit number encoded in two qubit, I can extend this to write a quantum program to compare two multiple-bit number.
   
Main idea used in comparison of two multiple-bit number: 
    Start comparing their leftmost bit and move to lower bit only if the compared bits are equal. Else the number with greater upper bit is greater than the other number.

References used:

Blogs: \
https://pyqml.medium.com/comparing-two-numbers-using-a-quantum-algorithm-22910cea56aa 
https://pyqml.medium.com/the-quantum-bit-comparator-463911f7bcd3 

Paper:
https://www.researchgate.net/publication/228574906_Quantum_bit_string_comparator_Circuits_and_applications#:~:text=This%20quantum%20circuit%20compares%20two,structure%20for%20designing%20of%20algorithms. 


# Compare Two 1-bit Binary Numbers

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.primitives.sampler import Sampler

**bit_compare** function compares two bits and tells us which one is greater by flipping one of two auxiliary qubits that correspond to the according qubit to one. 
If both numbers are equal, none of the auxiliary qubits is flipped to one.

In [ ]:
def bit_compare():
    qr = QuantumRegister(2, "bits")
    aux = QuantumRegister(2, "aux")
    
    qc = QuantumCircuit(qr, aux)
    qc.x(qr[1])
    qc.mcx(qr, aux[0])
    qc.x(qr[0])
    qc.x(qr[1])
    qc.mcx(qr, aux[1])
    qc.x(qr[0])
    
    return qc

bit_compare().draw("mpl")

**encode**: encodes the number in qubits of the circuit to enable comparison 

In [ ]:
def encode(number):
    qr = QuantumRegister(1, "number")
    qc = QuantumCircuit(qr)
    if (number == 1):
        qc.x(qr[0])
    return qc

**compare**: Quantum circuit that compares two number by encoding them and then using bit_compare to return the results of their comparison

In [ ]:
def compare(a, b):
    qra = QuantumRegister(1, "a")
    qrb = QuantumRegister(1, "b")
    qraux = QuantumRegister(2, "aux")

    qc = QuantumCircuit(qra, qrb, qraux)

    qc.append(encode(a), [*qra])
    qc.append(encode(b), [*qrb])

    qc.append(bit_compare(), [*qra, *qrb, *qraux])

    # Use AerSimulator
    qc_measured_this = qc.measure_all(inplace=False)
    sampler = Sampler()
    job = sampler.run(qc_measured_this, shots=1000)
    counts = job.result()
    
    return counts

In [ ]:
counts = compare(1,0)
plot_histogram(counts.quasi_dists[0].binary_probabilities())

# Compare Two n-bit strings 

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
from qiskit.primitives.sampler import Sampler

def bit_compare():
    qr = QuantumRegister(2, "bits")
    aux = QuantumRegister(2, "aux")
    
    qc = QuantumCircuit(qr, aux)
    qc.x(qr[1])
    qc.mcx(qr, aux[0])
    qc.x(qr[0])
    qc.x(qr[1])
    qc.mcx(qr, aux[1])
    qc.x(qr[0])
    
    return qc

In encode, we expect the provided bit to be a string, not a number.

In [ ]:
def encode(bit):
    qr = QuantumRegister(1, "number")
    qc = QuantumCircuit(qr)
    if (bit == "1"):
        qc.x(qr[0])
    return qc

The encode function takes a single-bit character. So, to encode an entire bitstring, we need to call this function multiple times.

**compare_bitstring**: 
1. First, we calculate the number of bits (qubits) we require to represent the numbers.
2. The qubits in the qra register represent the first number and the qubits in qrb the second accordingly.
3. qraux contains the auxiliary qubits we need to compare single digits. Since we need two of them to compare a single digit, we need 2*bits of them.
4. The qrint register further contains “intermediate” qubits that we need to connect the results of comparing single digits.
5. For each digit in our bitstrings, we encode the respective position of both bitstrings
6. We append the bit_compare function to compare these two digits using two qubits from the qraux register.
7. In the next block, we apply the following gates for each digit except the last. We apply x gates on both auxiliaries followed by a multi-controlled X gate (mcx) with the auxiliary qubits as control qubits and an “intermediate” qubit as the target.
8. We apply MCX gate only if qrint[-i] is 1, which happens only if ith digit of both bitstring number are equal 

In [ ]:
# Create circuit with qubits for bitstring a and b + Auxs for caclulation
def compare_bitstring(bitstring_a, bitstring_b, exec=True):
    bits = len(bitstring_a)
    qra = QuantumRegister(bits, "a")
    qrb = QuantumRegister(bits, "b")
    qraux = QuantumRegister(2*bits, "aux")
    qrint = QuantumRegister(bits-1, "int")
    
    # If using Sampler:
    # cr = ClassicalRegister(5*bits-1) 
    # Else:
    cr = ClassicalRegister(2)

    print(bits)
    
    qc = QuantumCircuit(qra, qrb, qraux, qrint, cr)

    # For each digit in our bitstrings, we encode the respective position of both bitstrings 
    # We append the bit_compare function 
    # to compare these two digits using two qubits from the qraux register.
    for i in range(bits):
        qc.append(encode(bitstring_a[i]), [qra[i]])
        qc.append(encode(bitstring_b[i]), [qrb[i]])
        qc.append(bit_compare(), [qra[i], qrb[i], qraux[2*i], qraux[2*i+1]])
        
        # We apply an x gate on the “intermediate” qubit only if both auxiliary qubits are 0.
        # This is only the case if the compared digits are equal.  
        if i < bits-1:
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
            qc.mcx([qraux[2*i], qraux[2*i+1]], qrint[i])
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
    
    # Applies MCX gate only if qrint[-i] is 1, 
    # which happens only if ith digit of both bitstring number are equal 
    for i in range(0, bits-1):
        qc.mcx([qrint[-i]], qraux[2*(-i-1)], qraux[2*(-i)])
        qc.mcx([qrint[-i]], qraux[2*(-i-1)+1], qraux[2*(-i)+1])
        
    # Only need to measure two qubits if not using the Sampler
    qc.measure(qraux[0], cr[0])
    qc.measure(qraux[1], cr[1])
    
    # Need to use measure_all since we are using Sampler for the circuit
    # qc.measure_all()
    
    if exec:
        # sampler = Sampler()
        # job = sampler.run(qc)
        # counts = job.result()
        
        simulator = AerSimulator()
        circ = transpile(qc, simulator)
        result = simulator.run(circ).result()
        counts = result.get_counts(circ)

        return counts
    else:
        return qc

### Rationale for : 
```
for i in range(0, bits-1):
        qc.mcx([qrint[-i]], qraux[2*(-i-1)], qraux[2*(-i)])
        qc.mcx([qrint[-i]], qraux[2*(-i-1)+1], qraux[2*(-i)+1])
```
##### First control qubit
First control qubit in both lines is qrint[-i]. 
Therefore, if qrint[-i] qubit is not 1 none of the following mcx gates will have any effect. Remember that this is only 1 if the previous digits are equal.

That is, we only care about lower digits if the upper digits are equal. We read bitstrings from right to left. So, the very first digit we read by iterating through the bitstring is the highest. If the two numbers we compare differ in that digit, we don’t need to look any further. For instance, 100 is greater than 011.

So, only if the upper digits are equal do we flip the corresponding “intermediate” qubit so that we “look” at the lower digits.

##### Second control qubit
The second control qubits of these gates are the auxiliary qubits of the lower comparison. One of these being one indicates that the corresponding digit is greater.

##### Target qubit
The target qubit of these mcx gates are the auxiliary qubits of the upper digit comparison. 

#### Conclusion
So, we look at the lower digits only if the upper digits are equal and the auxiliary qubits are 0. Then, if one of the lower digits is greater than the other, the corresponding mcx gate flips the auxiliary qubit of the upper digit comparison from 0 to 1. 

Therfore, to see which number is greater, we only need to look at the auxiliary qubits representing the upper digits.

### Final Quantum Program to compare two arbitary length bitstrings

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
from qiskit.primitives.sampler import Sampler

def bit_compare():
    qr = QuantumRegister(2, "bits")
    aux = QuantumRegister(2, "aux")
    
    qc = QuantumCircuit(qr, aux)
    qc.x(qr[1])
    qc.mcx(qr, aux[0])
    qc.x(qr[0])
    qc.x(qr[1])
    qc.mcx(qr, aux[1])
    qc.x(qr[0])
    
    return qc

def encode(bit):
    qr = QuantumRegister(1, "number")
    qc = QuantumCircuit(qr)
    if (bit == "1"):
        qc.x(qr[0])
    return qc

def compare_bitstring(bitstring_a, bitstring_b, exec=True):
    num_bits = len(bitstring_a)
    qra = QuantumRegister(num_bits, "a")
    qrb = QuantumRegister(num_bits, "b")
    qraux = QuantumRegister(2*num_bits, "aux")
    qrint = QuantumRegister(num_bits-1, "int")
    cr = ClassicalRegister(2)

    qc = QuantumCircuit(qra, qrb, qraux, qrint, cr)
    
    for i in range(num_bits):
        qc.append(encode(bitstring_a[i]), [qra[i]])
        qc.append(encode(bitstring_b[i]), [qrb[i]])
        qc.append(bit_compare(), [qra[i], qrb[i], qraux[2*i+1], qraux[2*i]])
        
        if i < num_bits-1:
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
            qc.mcx([qraux[2*i], qraux[2*i+1]], qrint[i])
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
        
    for i in range(0, num_bits-1):
        qc.mcx([qrint[-i]], qraux[2*(-i-1)], qraux[2*(-i)])
        qc.mcx([qrint[-i]], qraux[2*(-i-1)+1], qraux[2*(-i)+1])
        
    qc.measure(qraux[0], cr[0])
    qc.measure(qraux[1], cr[1])
    
    if exec:
        simulator = AerSimulator()
        circ = transpile(qc, simulator)
        result = simulator.run(circ).result()
        counts = result.get_counts(circ)
        return counts
    else:
        return qc

**NOTE:** We get '01' if bitstring_a is less than bitstring_b

In [ ]:
compare_bitstring('10','11', exec=False).draw("mpl")
compare_bitstring("01", "10")

This algorithm works for bitstrings of arbitrary size

# Less than K

Now, onto the screening task 1. \
Task 1: 
Given a positive integer “k” and a list of integer numbers, look for the numbers within the list, that are less than k. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case 


In [ ]:
def adjust_bitstrings(bitstring_a, bitstring_b):
    len_a = len(bitstring_a)
    len_b = len(bitstring_b)
        
    # Determine the maximum length
    max_len = max(len_a, len_b)
    
    # Add leading zeros to the shorter bitstring
    if len_a < max_len:
        bitstring_a = bitstring_a[2:].zfill(max_len-len_a)
    if len_b < max_len:
        bitstring_b = bitstring_b[2:].zfill(max_len-len_b)

    return bitstring_a, bitstring_b

In [ ]:
def less_than_k(k: int, list_n: list[int]):
    list_less_than_k = []
    bitstring_a = bin(k)
    for number in list_n:
        bitstring_b = bin(number)
        
        if len(bitstring_a) != len(bitstring_b):
            bitstring_a, bitstring_b = adjust_bitstrings(bitstring_a, bitstring_b)
        
        # Get the comparison of two numbers from our quantum program 
        count = compare_bitstring(bitstring_a, bitstring_b)
    
        # If number is less than k, then we get '01' 
        # and append that number to our list
        if count.get('01') is not None:
            list_less_than_k.append(number)
            
    return list_less_than_k

In [ ]:
A = less_than_k(7,[4,9,11,14,1,13,6,15])
print(A)

# for k=7 and number = 14:
# interestingly, the count from qProgram is '00' 
# because adding a trailing zero to bin(7) is equal to bin(14) 

Thus, we have the solution for task 1 